# Dang Thanh Vu - 197796

In [1]:
import numpy as np
import itertools

X1 = np.loadtxt('testClass1.dat')
X2 = np.loadtxt('testClass2.dat')

def fisher(X1, X2):
    m1 = np.mean(X1)
    m2 = np.mean(X2)
    var1 = np.var(x1)
    var2 = np.var(x2)
    return ((m1 - m2)**2)/(var1+var2)

def scatter(X1, X2, P):
    X = np.concatenate([X1, X2])
    mf0 = np.mean(X, axis=0)
    mf1 = np.mean(X1, axis=0)
    mf2 = np.mean(X2, axis=0)
    Sw = P[0]*np.cov(X1.T) + P[1]*np.cov(X2.T)
    Sb = P[0]*(np.array([mf1 - mf0]).T @ np.array([mf1 - mf0])) + P[1]*(np.array([mf2 - mf0]).T @ np.array([mf2 - mf0]))
    if(X.shape[1] == 1):
        Sw = np.array([[Sw]])
    J3 = np.trace(np.linalg.inv(Sw) @ (Sb + Sw))/X1.shape[1]
    return J3

def divergence(X1, X2):
    m, n = X1.shape[:]
    m1 = np.mean(X1, axis=0)
    m2 = np.mean(X2, axis=0)
    S1 = np.cov(X1.T)
    S2 = np.cov(X2.T)
    d = np.trace((np.linalg.inv(S1) @ S2) + (np.linalg.inv(S2) @ S1) - 2*np.identity(n))
    d = d + (np.array([m1-m2]) @ (np.linalg.inv(S1) + np.linalg.inv(S2)) @ np.array([m1-m2]).T)
    return d/(2*n)

def Bhattacharyya(X1, X2):
    m, n = X1.shape[:]
    m1 = np.mean(X1, axis=0)
    m2 = np.mean(X2, axis=0)
    S1 = np.cov(X1.T)
    S2 = np.cov(X2.T)
    A = np.log(np.linalg.det((S1 + S2)/2)/(np.sqrt(np.linalg.det(S1)*np.linalg.det(S2))))
    B = np.array([m1-m2]) @ np.linalg.inv((S1 + S2)/2) @ np.array([m1-m2]).T
    return (B/8 + A/2)/n

def ChernoffBound(X1, X2, P):
    Bh = Bhattacharyya(X1, X2)
    e = np.exp(-Bh)*np.sqrt(P[0]*P[1])
    return e

def Scalar_ranking(X1, X2, a1, a2):
    m, n = X1.shape[:]
    m1 = np.mean(X1, axis=0)
    m2 = np.mean(X2, axis=0)
    var1 = np.var(X1, axis=0)
    var2 = np.var(X2, axis=0)
    #fisher
    fisher = ((m1 - m2)**2)/(var1+var2)
    sort_fisher = np.sort(fisher)[::-1]
    first_rank = np.argsort(fisher)[::-1]
    rank_index = []
    rank_index.append(first_rank[0])
    X = np.concatenate([X1, X2])
    cov = np.cov(X.T)
    cov[rank_index[0]] = 0
    fisher[rank_index[0]] = -1000
    for i in range(1, n):
        rho = np.sum(cov[:,rank_index], axis=1)
        f = a1*fisher - a2/i*np.absolute(rho)
        index = np.argmax(f)
        cov[index] = 0
        fisher[index] = -1000
        rank_index.append(index)
        
    return rank_index

def exhautiveSearch(X1, X2, n_features, rank, l):
    m, n = X1.shape[:]
    coms = list(itertools.combinations(rank[:l], n_features))
    J3_measures = []
    for com in coms:
        Xf1 = X1[:, com]
        Xf2 = X2[:, com]
        J3_measures.append(scatter(Xf1, Xf2, np.array([0.5, 0.5])))
    id_max = np.argmax(np.array(J3_measures))
    return coms[id_max]

def backwardSearch(X1, X2, n_features, rank, l):
    m, n = X1.shape[:]
    i = l
    com = rank[:l]
    while(i > n_features):
        com = exhautiveSearch(X1, X2, i - 1, com, i)
        i = i - 1
        print(com)
    return com

def forwardSearch(X1, X2, n_features, rank, l):
    m, n = X1.shape[:]
    i = 1
    com = [rank[0]]
    while(len(com) < n_features):
        J3 = 0
        for k in rank[:l]:
            if k not in com:
                temp = com[:]
                temp.append(k)
                Xf1 = X1[:, temp]
                Xf2 = X2[:, temp]
                J3_measures = (scatter(Xf1, Xf2, np.array([0.5, 0.5])))
                if(J3_measures > J3):
                    index = k
                    J3 = J3_measures
        com.append(index)
        print(com)
    return com

def floatingSearch(X1, X2, n_features, rank, l):
    m, n = X1.shape[:]
    i = 1
    com = [rank[0]]
    while(len(com) <= n_features):
        #add
        J3 = 0
        for k in rank[:l]:
            if k not in com:
                temp = com[:]
                temp.append(k)
                Xf1 = X1[:, temp]
                Xf2 = X2[:, temp]
                J3_measures = (scatter(Xf1, Xf2, np.array([0.5, 0.5])))
                if(J3_measures > J3):
                    pos = k
                    index = k
                    J3 = J3_measures
        com.append(index)
        #remove
        for h in com:
            temp = com[:]
            temp.remove(h)
            Xf1 = X1[:, temp]
            Xf2 = X2[:, temp]
            J3_measures = (scatter(Xf1, Xf2, np.array([0.5, 0.5])))
            if(J3_measures > J3):
                index = h
                J3 = J3_measures
        if(index != pos):
            com.remove(index)
        print(com)
    return com[:n_features]

In [2]:
X = np.concatenate([X1, X2])
m0 = np.mean(X, axis=0)
var = np.var(X, axis=0)
Xn = (X - m0)/(np.sqrt(var))
Xf1 = Xn[:25]
Xf2 = Xn[25:]

rank = Scalar_ranking(Xf1, Xf2, 0.2, 0.8)
print(rank)

[4, 7, 5, 6, 1, 0, 16, 8, 10, 19, 18, 17, 2, 3, 11, 12, 15, 9, 14, 13]


In [3]:
com = exhautiveSearch(Xf1, Xf2, 2, rank, 14)
print("exhaust - best combination", com)
com = backwardSearch(Xf1, Xf2, 2, rank, 14)
print("backward - best combination", com)
com = forwardSearch(Xf1, Xf2, 2, rank, 14)
print("forward - best combination", com)
com = floatingSearch(Xf1, Xf2, 2, rank, 14)
print("floating - best combination", com)

exhaust - best combination (5, 0)
(7, 5, 6, 1, 0, 16, 8, 10, 19, 18, 17, 2, 3)
(7, 5, 6, 1, 0, 8, 10, 19, 18, 17, 2, 3)
(7, 5, 6, 1, 0, 8, 10, 19, 17, 2, 3)
(7, 5, 6, 1, 0, 8, 19, 17, 2, 3)
(7, 5, 1, 0, 8, 19, 17, 2, 3)
(7, 5, 1, 0, 8, 17, 2, 3)
(7, 5, 1, 0, 8, 2, 3)
(5, 1, 0, 8, 2, 3)
(1, 0, 8, 2, 3)
(1, 0, 8, 3)
(1, 0, 8)
(1, 8)
backward - best combination (1, 8)
[4, 0]
forward - best combination [4, 0]
[4, 0]
[0, 5]
[0, 5, 8]
floating - best combination [0, 5]


In [4]:
com = exhautiveSearch(Xf1, Xf2, 2, rank, 4)
print("exhaust - best combination", com)
com = backwardSearch(Xf1, Xf2, 2, rank, 4)
print("backward - best combination", com)
com = forwardSearch(Xf1, Xf2, 2, rank, 4)
print("forward - best combination", com)
com = floatingSearch(Xf1, Xf2, 2, rank, 4)
print("floating - best combination", com)

exhaust - best combination (4, 5)
(4, 5, 6)
(4, 5)
backward - best combination (4, 5)
[4, 5]
forward - best combination [4, 5]
[4, 5]
[4, 5, 6]
floating - best combination [4, 5]


In [5]:
com = exhautiveSearch(Xf1, Xf2, 2, rank, 8)
print("exhaust - best combination", com)
com = backwardSearch(Xf1, Xf2, 2, rank, 8)
print("backward - best combination", com)
com = forwardSearch(Xf1, Xf2, 2, rank, 8)
print("forward - best combination", com)
com = floatingSearch(Xf1, Xf2, 2, rank, 8)
print("floating - best combination", com)

exhaust - best combination (5, 0)
(4, 7, 5, 6, 1, 0, 8)
(4, 7, 5, 1, 0, 8)
(4, 5, 1, 0, 8)
(5, 1, 0, 8)
(1, 0, 8)
(1, 8)
backward - best combination (1, 8)
[4, 0]
forward - best combination [4, 0]
[4, 0]
[0, 5]
[0, 5, 8]
floating - best combination [0, 5]


In [6]:
com = exhautiveSearch(Xf1, Xf2, 2, rank, 10)
print("exhaust - best combination", com)
com = backwardSearch(Xf1, Xf2, 2, rank, 10)
print("backward - best combination", com)
com = forwardSearch(Xf1, Xf2, 2, rank, 10)
print("forward - best combination", com)
com = floatingSearch(Xf1, Xf2, 2, rank, 10)
print("floating - best combination", com)

exhaust - best combination (5, 0)
(7, 5, 6, 1, 0, 16, 8, 10, 19)
(7, 5, 6, 1, 0, 8, 10, 19)
(7, 5, 6, 1, 0, 8, 10)
(7, 5, 6, 1, 0, 8)
(7, 5, 1, 0, 8)
(5, 1, 0, 8)
(1, 0, 8)
(1, 8)
backward - best combination (1, 8)
[4, 0]
forward - best combination [4, 0]
[4, 0]
[0, 5]
[0, 5, 8]
floating - best combination [0, 5]


In [7]:
com = exhautiveSearch(Xf1, Xf2, 2, rank, 20)
print("exhaust - best combination", com)
com = backwardSearch(Xf1, Xf2, 2, rank, 20)
print("backward - best combination", com)
com = forwardSearch(Xf1, Xf2, 2, rank, 20)
print("forward - best combination", com)
com = floatingSearch(Xf1, Xf2, 2, rank, 20)
print("floating - best combination", com)

exhaust - best combination (5, 0)
(4, 7, 5, 6, 1, 0, 16, 8, 10, 19, 17, 2, 3, 11, 12, 15, 9, 14, 13)
(4, 7, 5, 1, 0, 16, 8, 10, 19, 17, 2, 3, 11, 12, 15, 9, 14, 13)
(4, 7, 5, 1, 0, 16, 8, 10, 19, 17, 2, 3, 11, 12, 15, 14, 13)
(4, 7, 1, 0, 16, 8, 10, 19, 17, 2, 3, 11, 12, 15, 14, 13)
(4, 7, 1, 0, 16, 8, 10, 19, 2, 3, 11, 12, 15, 14, 13)
(4, 7, 1, 0, 8, 10, 19, 2, 3, 11, 12, 15, 14, 13)
(4, 7, 1, 0, 8, 10, 19, 2, 3, 11, 12, 15, 13)
(4, 7, 1, 0, 8, 10, 19, 2, 3, 11, 15, 13)
(4, 7, 1, 0, 8, 10, 2, 3, 11, 15, 13)
(4, 1, 0, 8, 10, 2, 3, 11, 15, 13)
(4, 1, 0, 8, 10, 2, 3, 15, 13)
(4, 1, 0, 8, 2, 3, 15, 13)
(1, 0, 8, 2, 3, 15, 13)
(1, 0, 8, 2, 3, 15)
(1, 0, 8, 2, 3)
(1, 0, 8, 3)
(1, 0, 8)
(1, 8)
backward - best combination (1, 8)
[4, 0]
forward - best combination [4, 0]
[4, 0]
[0, 5]
[0, 5, 8]
floating - best combination [0, 5]


|n       | 4    |8    |10    |14    |20    |
|--------|------|-----|------|------|------|
|exhaust |(4,5) |(5,0)|(5,0) |(5,0) |(5,0) |
|forward |(4,5) |(4,0)|(4,0) |(4,0) |(4,0) |
|backward|(4,5) |(1,8)|(1,8) |(1,8) |(1,8) |
|floating|(4,5) |(0,5)|(0,5) |(0,5) |(0,5) |